# Dash

## Load libraries


In [19]:
import importlib

import os
import numpy as np
import time

import torch
from torch import nn
import torch.optim as optim
from torch.utils import data
from torchvision import transforms
import distiller.apputils as apputils

import sys

import ai8x

kws20 = importlib.import_module("datasets.kws20")

## Calculate weights

In [2]:
from pathlib import Path

raw_data_path = Path('data/KWS_DASH/raw')
class_file_count = {}

class_dirs = [d for d in raw_data_path.iterdir() if d.is_dir() and d.stem != "_background_noise_"]


for d in class_dirs:
    class_file_count[d] = len(list(d.iterdir()))

min_file_count = float(min(class_file_count.values()))

for d in class_dirs:
    class_file_count[d] = min_file_count / class_file_count[d]
    print(f"{d.stem}: {round(class_file_count[d], 7)}")

blinds: 1.0
dash: 1.0
down: 0.0079142
energy: 0.96875
lights: 0.96875
off: 0.0082777
on: 0.0080624
up: 0.0083266


## Generate processed dataset

In [3]:
train_batch_size = 128
train_loader, val_loader, test_loader, _ = apputils.get_data_loaders(
    kws20.KWS_DASH_get_datasets, ("data", False), train_batch_size, 1, validation_split=0)
print(f"Dataset sizes:\n\ttraining={len(train_loader.sampler)}\n\tvalidation={len(val_loader.sampler)}\n\ttest={len(test_loader.sampler)}")

No key `noise_var` in input augmentation dictionary!  Using defaults: [Min: 0., Max: 1.]
No key `shift` in input augmentation dictionary! Using defaults: [Min:-0.1, Max: 0.1]
No key `strech` in input augmentation dictionary! Using defaults: [Min: 0.8, Max: 1.3]

Processing train...
Class blinds (# 0): 81 elements
Class dash (# 1): 81 elements
Class down (# 2): 10665 elements
Class energy (# 3): 84 elements
Class lights (# 4): 84 elements
Class off (# 5): 10158 elements
Class on (# 6): 10464 elements
Class up (# 7): 10152 elements
Class UNKNOWN: 0 elements

Processing test...
Class blinds (# 0): 12 elements
Class dash (# 1): 12 elements
Class down (# 2): 1086 elements
Class energy (# 3): 12 elements
Class lights (# 4): 12 elements
Class off (# 5): 1077 elements
Class on (# 6): 1071 elements
Class up (# 7): 1017 elements
Class UNKNOWN: 0 elements
Dataset sizes:
	training=41769
	validation=4299
	test=4299


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [5]:
classes = ['blinds', 'dash', 'down', 'energy', 'lights', 'off', 'on', 'up']

## Load pretrained model


In [6]:
def count_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

In [24]:
ai8x.set_device(device=85, simulate=False, round_avg=False)

mod = importlib.import_module("models.ai85net-kws20-v3")

model = mod.AI85KWS20Netv3(num_classes=21, num_channels=128, dimensions=(128, 1), bias=False)
print(f'Number of Model Params: {count_params(model)}')

model, compression_scheduler, optimizer, start_epoch = apputils.load_checkpoint(
            model, "logs/2022.05.20-235449/qat_best.pth.tar", model_device='cuda')



Configuring device: MAX78000, simulate=False.
Number of Model Params: 169472


## Replace FC layer and freeze the rest of the layers

In [8]:
def freeze_layer(layer):
    for p in layer.parameters():
        p.requires_grad = False

In [25]:
freeze_layer(model.voice_conv1)
freeze_layer(model.voice_conv2)
freeze_layer(model.voice_conv3)
freeze_layer(model.voice_conv4)
freeze_layer(model.kws_conv1)
# freeze_layer(model.kws_conv2)
# freeze_layer(model.kws_conv3)
# freeze_layer(model.kws_conv4)
model.fc = ai8x.Linear(256, 8, bias=False, wide=True)

model = model.to(device)

## Train the model

In [26]:
num_epochs = 50
epoch = 0
optimizer = optim.Adam(model.parameters(), lr=0.0001)
ms_lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 80], gamma=0.5)
criterion = torch.nn.CrossEntropyLoss(
    torch.Tensor((1, 1, 0.01, 1, 1, 0.01, 0.01, 0.01))
)
criterion.to(device)

qat_policy = {
    'start_epoch': 20,
    'weight_bits': 8
    }

In [11]:
from sklearn.metrics import confusion_matrix

def plot_confusion(y_true, y_pred, classes):
    cf_matrix = confusion_matrix(y_true, y_pred, list(range(len(classes))))
    print(cf_matrix)


In [27]:
best_acc = 0
best_qat_acc = 0
for epoch in range(0, num_epochs):
    if epoch > 0 and epoch == qat_policy['start_epoch']:
        print('QAT is starting!')
        # Fuse the BN parameters into conv layers before Quantization Aware Training (QAT)
        ai8x.fuse_bn_layers(model)

        # Switch model from unquantized to quantized for QAT
        ai8x.initiate_qat(model, qat_policy)

        # Model is re-transferred to GPU in case parameters were added
        model.to(device)
    running_loss = []
    train_start = time.time()
    model.train()
    for idx, (inputs, target) in enumerate(train_loader):
        inputs = inputs.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        
        model_out = model(inputs)
        
        loss = criterion(model_out, target)
        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())

    mean_loss = np.mean(running_loss)
    train_end = time.time()
    print("Epoch: {}/{}\t LR: {}\t Train Loss: {:.4f}\t Dur: {:.2f} sec.".format(
        epoch+1, num_epochs, ms_lr_scheduler.get_lr(), mean_loss, (train_end-train_start)))
    
    model.eval()
    acc = 0.
    acc_weight = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs, target in test_loader:
            inputs = inputs.to(device)
            target = target.to(device)
            model_out = model(inputs)
            target_out = torch.argmax(model_out, dim=1)
            
            y_pred.extend(target_out.cpu().numpy())
            y_true.extend(target.cpu().numpy())
            
            tp = torch.sum(target_out == target)
            acc_batch = (tp / target_out.numel()).detach().item()
            acc += target_out.shape[0] * acc_batch
            acc_weight += target_out.shape[0]
            
        total_acc = 100 * (acc / acc_weight)
        if epoch == qat_policy['start_epoch']: best_acc = 0
        if total_acc > best_acc:
            best_acc = total_acc
            checkpoint_extras = {'current_top1': best_acc,
                                 'best_top1': best_acc,
                                 'best_epoch': epoch}
            model_name = 'ai85net_kws_dash'
            model_prefix = f'{model_name}' if epoch < qat_policy['start_epoch'] else (f'qat_{model_name}')
            apputils.save_checkpoint(epoch, model_name, model, optimizer=optimizer,
                                     scheduler=None, extras=checkpoint_extras,
                                     is_best=True, name=model_prefix,
                                     dir='.')
            print(f'Best model saved with accuracy: {best_acc:.2f}%')
            
        print('\t\t Test Acc: {:.2f}'.format(total_acc))
        print("\t\tConfusion:")
        plot_confusion(y_true, y_pred, classes)
    ms_lr_scheduler.step()

Epoch: 1/50	 LR: [0.0001]	 Train Loss: 1.6892	 Dur: 36.19 sec.
Best model saved with accuracy: 86.28%
		 Test Acc: 86.28
		Confusion:
[[   0    0    1    0    7    0    4    0]
 [   0    0    4    0    2    4    0    2]
 [   0    1 1039    0    1   12   13   20]
 [   0    0    0    0    0    0    7    5]
 [   0    0    1    0    7    0    2    2]
 [   0    0   31    0    0  929   50   67]
 [   0    0   13    0    0   71  972   15]
 [   0    0   15    0    0  192   48  762]]
Epoch: 2/50	 LR: [0.0001]	 Train Loss: 1.1945	 Dur: 24.38 sec.
Best model saved with accuracy: 88.28%
		 Test Acc: 88.28
		Confusion:
[[   3    1    1    0    6    1    0    0]
 [   1    6    2    0    1    1    0    1]
 [   0    6 1045    2    1   10   10   12]
 [   0    2    0    5    0    0    2    3]
 [   0    2    1    0    8    0    0    1]
 [   2    2   22    0    0  916   50   85]
 [   1    0   10    1    0   61  978   20]
 [   1    2   13    3    0  127   37  834]]
Epoch: 3/50	 LR: [0.0001]	 Train Loss: 0.9

## Quantize model in the sythesis repo!!

## Evaluate quantized model

In [29]:
selected_model = '../ai8x-synthesis/trained/ai85-kws-dash-qat8-q.pth.tar'

ai8x.set_device(device=85, simulate=True, round_avg=False)

model = mod.AI85KWS20Netv3(num_classes=8, num_channels=128, dimensions=(128, 1), bias=False)
print(f'Number of Model Params: {count_params(model)}')
ai8x.fuse_bn_layers(model)
model, compression_scheduler, optimizer, start_epoch = apputils.load_checkpoint(
            model, selected_model, model_device=device)
ai8x.update_model(model)
model = model.to(device)

Configuring device: MAX78000, simulate=True.
Number of Model Params: 166144


In [30]:
train_batch_size = 128
train_loader, val_loader, test_loader, _ = apputils.get_data_loaders(
    kws20.KWS_DASH_get_datasets, ("data", True), train_batch_size, 1, validation_split=0)
print(f"Dataset sizes:\n\ttraining={len(train_loader.sampler)}\n\tvalidation={len(val_loader.sampler)}\n\ttest={len(test_loader.sampler)}")

No key `noise_var` in input augmentation dictionary!  Using defaults: [Min: 0., Max: 1.]
No key `shift` in input augmentation dictionary! Using defaults: [Min:-0.1, Max: 0.1]
No key `strech` in input augmentation dictionary! Using defaults: [Min: 0.8, Max: 1.3]

Processing train...
Class blinds (# 0): 81 elements
Class dash (# 1): 81 elements
Class down (# 2): 10665 elements
Class energy (# 3): 84 elements
Class lights (# 4): 84 elements
Class off (# 5): 10158 elements
Class on (# 6): 10464 elements
Class up (# 7): 10152 elements
Class UNKNOWN: 0 elements

Processing test...
Class blinds (# 0): 12 elements
Class dash (# 1): 12 elements
Class down (# 2): 1086 elements
Class energy (# 3): 12 elements
Class lights (# 4): 12 elements
Class off (# 5): 1077 elements
Class on (# 6): 1071 elements
Class up (# 7): 1017 elements
Class UNKNOWN: 0 elements
Dataset sizes:
	training=41769
	validation=4299
	test=4299


In [31]:
model.eval()
acc = 0.
acc_weight = 0
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, target in test_loader:
        inputs = inputs.to(device)
        target = target.to(device)
        model_out = model(inputs)
        target_out = torch.argmax(model_out, dim=1)
        
        y_pred.extend(target_out.cpu().numpy())
        y_true.extend(target.cpu().numpy())
        
        tp = torch.sum(target_out == target)
        acc_batch = (tp / target_out.numel()).detach().item()
        acc += target_out.shape[0] * acc_batch
        acc_weight += target_out.shape[0]
    
    total_acc = 100 * (acc / acc_weight)

print('Confusion:')
plot_confusion(y_true, y_pred, classes)
print(f'Quantized accuracy: {total_acc:.2f}%')

Confusion:
[[  8   0   0   1   0   0   3   0]
 [  6   0   4   1   0   0   1   0]
 [108  30 765  34   5  46  36  62]
 [  3   1   4   1   1   0   1   1]
 [  5   0   0   1   0   2   3   1]
 [ 48   6  18   9   1 757 131 107]
 [ 62  10  17   9   0 236 657  80]
 [ 26   7  18  11   1 173  71 710]]
Quantized accuracy: 67.41%
